In [2]:
import os
import sys

current_path = os.getcwd()
current_path = os.path.join(current_path, 'Dataset')
print(current_path)
folder_com = os.path.join(current_path, 'Commercial Aero')
folder_uss = os.path.join(current_path, 'USS')

def get_file_as_id(folder_path):
    if not os.path.exists(folder_path):
        return ValueError
    folder_name = folder_path.split('/')[-1]
    id = folder_name.split('.')[0]
    id = id.split('_')[1:]
    id = ' '.join(id)

    return id

for file in os.listdir(folder_com):
    print(get_file_as_id(os.path.join(folder_com, file)))
    

/home/user/AI-Agents-For-ST/Dataset
CabinInterior JAMCO 2022
CompositePanels AirAsiaX 2022
General SpiritAeroSystems 2021
AirframeServices MTU 2022
AAM AerSaleCorp 2022
AirframeServices MTU 2021
CompositePanels ParkAerospace 2022
AAM OrixCorp 2022
General Airbus 2021
Generall AARCorp 2022
AAM AirTransportServices 2021
Generall AARCorp 2023
General Boeing 2022
PrecisionManufacturing ImproPrecision 2021
Nacelles TriumphGroup 2023
CabinInterior DroneDelivery 2021
AAM BOCAviation 2021
CabinInterior Coretronic 2022
CompositePanels Tejin 2022
Nacelles TriumphGroup 2022
EngineServices Safran 2021
CompositePanels AirAsiaX 2021
General Airbus 2022
AAM AirLeaseCorp 2022
CompositePanels Bombardier 2021
AAM WillisLease 2022
CabinInterior ParrotSA 2021
CabinInterior DroneDelivery 2022
General Boeing 2021
EngineServices DeutscheLufhansa 2021
EngineServices DeutscheLufhansa 2022
AAM AerSaleCorp 2023
CompositePanels Tejin 2023
CompositePanels ParkAerospace 2021
General SpiritAeroSystems 2022
Component

In [5]:
from dsrag.knowledge_base import KnowledgeBase
from dsrag.llm import OpenAIChatAPI
from dsrag.reranker import NoReranker, CohereReranker

from dsrag.database.vector.chroma_db import ChromaDB
from dsrag.document_parsing import extract_text_from_pdf

llm = OpenAIChatAPI(model='gpt-4o-mini')
reranker = CohereReranker()

com_id = "CommAero"
uss_id = "USS"

def create_kb(kb_id, folder_path):
    Kb = KnowledgeBase(kb_id, reranker = CohereReranker(), vector_db=ChromaDB(kb_id), storage_directory="~/AI-Agents-For-ST/storage")
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        id = get_file_as_id(file_path)
        print(id)
        text = extract_text_from_pdf(file_path)
        Kb.add_document(doc_id=id, text=text[0])
    return Kb

com_kb = create_kb(com_id, folder_com)
uss_kb = create_kb(uss_id, folder_uss)

# com_kb = KnowledgeBase(com_id, reranker = NoReranker(), vector_db=ChromaDB(kb_id), storage_directory="~/AI-Agents-For-ST/storage")

CabinInterior JAMCO 2022
Document with ID CabinInterior JAMCO 2022 already exists in the KB. Skipping...
CompositePanels AirAsiaX 2022
Document with ID CompositePanels AirAsiaX 2022 already exists in the KB. Skipping...
General SpiritAeroSystems 2021
Document with ID General SpiritAeroSystems 2021 already exists in the KB. Skipping...
AirframeServices MTU 2022
Document with ID AirframeServices MTU 2022 already exists in the KB. Skipping...
AAM AerSaleCorp 2022
Document with ID AAM AerSaleCorp 2022 already exists in the KB. Skipping...
AirframeServices MTU 2021
Document with ID AirframeServices MTU 2021 already exists in the KB. Skipping...
CompositePanels ParkAerospace 2022
Document with ID CompositePanels ParkAerospace 2022 already exists in the KB. Skipping...
AAM OrixCorp 2022
Document with ID AAM OrixCorp 2022 already exists in the KB. Skipping...
General Airbus 2021
Document with ID General Airbus 2021 already exists in the KB. Skipping...
Generall AARCorp 2022
Document with ID Ge

In [9]:
from dsrag.knowledge_base import KnowledgeBase
from dsrag.llm import OpenAIChatAPI
from dsrag.reranker import NoReranker, CohereReranker

from dsrag.database.vector.chroma_db import ChromaDB
from dsrag.document_parsing import extract_text_from_pdf

llm = OpenAIChatAPI(model='gpt-4o-mini')
reranker = CohereReranker()

com_id = "CommAero"
uss_id = "USS"

def Kb_add_doc(Kb, file_path):

    id = get_file_as_id(file_path)
    print(id)
    text = extract_text_from_pdf(file_path)
    Kb.add_document(doc_id=id, text=text[0])
    return Kb

com_kb = KnowledgeBase(com_id, reranker = CohereReranker(), vector_db=ChromaDB(com_id), storage_directory="~/AI-Agents-For-ST/storage")
uss_kb = KnowledgeBase(uss_id, reranker = CohereReranker(), vector_db=ChromaDB(uss_id), storage_directory="~/AI-Agents-For-ST/storage")


# com_kb = KnowledgeBase(com_id, reranker = NoReranker(), vector_db=ChromaDB(kb_id), storage_directory="~/AI-Agents-For-ST/storage")

In [10]:
file = "Stuck folders/CommAero_AAM_WillisLease_2022.pdf"

Kb_add_doc(com_kb, file)

AAM WillisLease 2022


/home/user/AI-Agents-For-ST/.venv/lib/python3.10/site-packages/instructor/client.py:396: DeprecationWarning: FUNCTIONS is deprecated and will be removed in future versions
  instructor.Mode.FUNCTIONS,
/home/user/AI-Agents-For-ST/.venv/lib/python3.10/site-packages/instructor/process_response.py:236: DeprecationWarning: FUNCTIONS is deprecated and will be removed in future versions
  if mode == Mode.FUNCTIONS:
/home/user/AI-Agents-For-ST/.venv/lib/python3.10/site-packages/instructor/function_calls.py:121: DeprecationWarning: FUNCTIONS is deprecated and will be removed in future versions
  if mode == Mode.FUNCTIONS:


Adding 636 chunks to the database


In [22]:
com_kb = KnowledgeBase(com_id, reranker = CohereReranker(), vector_db=ChromaDB(com_id), storage_directory="~/AI-Agents-For-ST/storage")
print(len(com_kb.chunk_db.data))
print(com_kb.chunk_db.data.keys())
print(len(com_kb.chunk_db.data['General SpiritAeroSystems 2021']))
# print(kb.chunk_db.data['AerSaleCorp_2023'])

# num_chunks = len(kb.chunk_db.data[doc_id])
# print (num_chunks)

# chunks = []
# for i in range(num_chunks):
#     chunk = {
#         "kb_id": kb_id,
#         "doc_id": doc_id,
#         "section_title": kb.chunk_db.get_section_title(doc_id, i),
#         "section_summary": kb.chunk_db.get_section_summary(doc_id, i),
#         "chunk_text": kb.chunk_db.get_chunk_text(doc_id, i),
#         "document_title": kb.chunk_db.get_document_title(doc_id, i),
#         "document_summary": kb.chunk_db.get_document_summary(doc_id, i)
#     }

#     chunks.append(chunk)

# print(chunks[0])

query = ["What is the revenue of General SpiritAeroSystems 2021"]
print(com_kb.query(query)[0]["text"])
# print(len(test.chunk_db.data))

3
dict_keys(['CabinInterior JAMCO 2022', 'CompositePanels AirAsiaX 2022', 'General SpiritAeroSystems 2021'])
985
Document context: the following excerpt is from a document titled 'Form 10-K Annual Report for Spirit AeroSystems Holdings, Inc. for the Fiscal Year Ended December 31, 2021'. This document is about: the annual financial performance and operational overview of Spirit AeroSystems Holdings, Inc. for the fiscal year ended December 31, 2021, including business segments, risk factors, and management's discussion and analysis.

both metallic and composite materials. For the twelve months ended December 31, 2021, we generated net revenues of $3,953.0 million and had net loss of$(540.8) million.
Operating Segments and Products
On
 September 30, 2021, we announced a new organizational structure to focus on growth in the key markets that we serve. Under the new structure that becameeffective
 during the fourth quarter of 2021, we operate in three principal segments: Commercial, Defense